_Cell 1: from jupyterlab template_: run it

In [1]:
import sys
from pathlib import Path
import time
import numpy as np
from pprint import pprint as pp
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
#plt.style.use('seaborn-v0_8-muted')

from IPython.display import Markdown
# To get multiple outputs into 1 cell w/o using print:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# autoreload extension
from IPython import get_ipython

ipython = get_ipython()
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

# -----------------------------------------
# TWO USEFUL FUNCTIONS:

def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    if up:
        newp = Path(this_path).parent
    else:
        newp = Path(this_path)
    src = newp.joinpath("src")
    if src.exists():
        newp = str(src)
    else:
        newp = str(newp)
    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))


# Filtered dir() for method discovery:
def fdir(obj, start_with_str='_', exclude=True):
    return [d for d in dir(obj) if not d.startswith(start_with_str) == exclude]


_Cell 2: from jupyterlab template_: run it

In [250]:
# Insert current src dir into sys.path so that modules in ../src can be imported:
# CHANGE THIS IF NEEDED:

add_to_sys_path(Path.cwd(), up=True)

In [255]:
DATA = Path.cwd().parent.joinpath("data")
if DATA.exists():
    DATA
else:
    print(f"Not found: {DATA}")

PosixPath('/home/cat/projects/mcce_dev/data')

In [260]:
mcce_dir = DATA.joinpath("4lzt")
mcce_dir
msout_dir = mcce_dir.joinpath("ms_out")
!ls -l {msout_dir}

runprm = mcce_dir.joinpath("run.prm.record")
runprm2 = runprm.relative_to(Path.cwd().parent)  # use when running from this nbk

PosixPath('/home/cat/projects/mcce_dev/data/4lzt')

total 92052
-rw-r--r-- 1 cat cat 46308419 Nov  2 17:03 pH6eH0ms.txt
-rw-r--r-- 1 cat cat 47949730 Nov  2 17:03 pH7eH0ms.txt


In [274]:
import ms_sampling_to_pdbs as sampling
import subprocess

In [276]:
#fdir(sampling)

---
# Test executable module

In [293]:
which = msout_dir
which = msout_dir.joinpath("pH6eH0ms")
!ls -l {which}

total 42672
-rw-r--r-- 1 cat cat 7271487 Nov  3 17:32 MC0
-rw-r--r-- 1 cat cat 7277473 Nov  3 17:32 MC1
-rw-r--r-- 1 cat cat 7272764 Nov  3 17:32 MC2
-rw-r--r-- 1 cat cat 7300876 Nov  3 17:32 MC3
-rw-r--r-- 1 cat cat 7283815 Nov  3 17:32 MC4
-rw-r--r-- 1 cat cat 7273831 Nov  3 17:32 MC5
-rw-r--r-- 1 cat cat    1372 Nov  3 17:32 header


In [281]:
mcce_dir = DATA.joinpath("4lzt")
mcce_dir
pH = 6.0
Eh= 0.0

PosixPath('/home/cat/projects/mcce_dev/data/4lzt')

In [349]:
ms = sampling.MS(mcce_dir,pH, Eh)

Getting runprm data.
Getting conformers data.
msout_fpath: /home/cat/projects/mcce_dev/data/4lzt/ms_out/pH6eH0ms.txt
msout_file_dir, created: False /home/cat/projects/mcce_dev/data/4lzt/ms_out/pH6eH0ms
Getting header data.
Getting MC data for: 0


In [347]:
#fdir(ms)

In [350]:
n_ires = len(ms.ires_by_iconf)
print("n_ires = len(ms.ires_by_iconf) = ", n_ires)
n_confs = len(ms.conformers)
print("n_confs = len(ms.conformers) = ", n_confs)
n_fixed = len(ms.fixed_iconfs)
print("n_fixed = len(ms.fixed_iconfs) = ", n_fixed)
free_confs = n_confs - n_fixed
print("free_confs = n_confs - n_fixed = ", free_confs)
n_free = len(ms.free_residues)
print("n_free = len(ms.free_residues) = ", n_free)

print(f"{ms.MC_RUNS = }, {ms.MC_NITER = :,}")
print(f"{ms.counts = :,}")

print(f"{ms.MC_NITER * ms.MC_RUNS * n_confs  = :,}")
print(f"{ms.MC_NITER * ms.MC_RUNS * free_confs = :,}")
print(f"{ms.MC_NITER * ms.MC_RUNS * n_ires = :,}")
print(f"{ms.MC_NITER * ms.MC_RUNS * n_fixed = :,}")
print(f"{ms.MC_NITER * ms.MC_RUNS * n_free = :,}")

n_ires = len(ms.ires_by_iconf) =  274
n_confs = len(ms.conformers) =  368
n_fixed = len(ms.fixed_iconfs) =  65
free_confs = n_confs - n_fixed =  303
n_free = len(ms.free_residues) =  63
ms.MC_RUNS = 6, ms.MC_NITER = 2,000
ms.counts = 550,000
ms.MC_NITER * ms.MC_RUNS * n_confs  = 4,416,000
ms.MC_NITER * ms.MC_RUNS * free_confs = 3,636,000
ms.MC_NITER * ms.MC_RUNS * n_ires = 3,288,000
ms.MC_NITER * ms.MC_RUNS * n_fixed = 780,000
ms.MC_NITER * ms.MC_RUNS * n_free = 756,000


In [360]:
n_counts = 0.0
ms_count_values = []

ms_list = ms.sort_microstates(by="energy")

for mc in ms_list:
    # n_counts += ms[1]
    # ms_count_values.append(ms[1])
    n_counts += mc.count
    ms_count_values.append(mc.count)

n_counts = ms.counts
ms_cumsum = np.cumsum([mc.count for mc in ms_list])

ms.counts
n_counts
ms_cumsum[-1]
ms_cumsum2[-1]
len(ms.microstates)
len(ms_count_values)

550000

550000.0

550000

550000

431705

431705

In [ ]:
def sample_microstates(
    ms: MS,
    size: int,
    kind: str = "deterministic",
    sort_by: str = "energy",
    reverse: bool = False,
) -> tuple:
    """
    Implement a sampling of MS.microstates depending on `kind`.
    Args:
        ms (MS): An instance of the MS class.
        size (int): sample size
        kind (str, 'deterministic'): Sampling kind: one of ['deterministic', 'random'].
            If 'deterministic', the microstates in ms_list are sorted then sampled at
            regular intervals otherwise, the sampling is random. Case insensitive.
        sort_by (str, "energy"): Only applies if kind is "deterministic".
        reverse (bool, False): Only applies if kind is "deterministic".
    Returns:
        A 3-tuple: cumsum of MC.count in ms.microstates,
                   array of indices for selection,
                   ms_list?
    """

    kind = kind.lower()
    if kind not in ["deterministic", "random"]:
        raise ValueError(
            f"Values for `kind` are 'deterministic' or 'random'; Given: {kind}"
        )
    if kind == "deterministic":
        if (sort_by is None) or (sort_by == "energy"):
            sort_by = "E"
        else:
            sort_by = sort_by.lower()
            if sort_by not in ["energy", "count"]:
                raise ValueError(f"Values for `sort_by` are 'energy' or 'count'; Given: {by}")

        ms_list = ms.sort_microstates(by=sort_by, reverse=reverse)
    else:
        ms_list = ms.microstates

    ms_cumsum = np.cumsum([mc.count for mc in ms_list])

    if kind == "deterministic":
        X = ms.counts - size
        Y = ms.counts / size
        count_selection = np.arange(size, X, Y)
    else:
        rng = np.random.default_rng()
        count_selection = rng.integers(low=1, high=n_counts + 1, size=size)

    return ms_cumsum, count_selection, ms_list

# Getters and Setters

---

---

# MCCE - MS Sampling (using test data in ../tests/data/)
---

In [3]:
import base
import mcce_io as io
import ms_sampling as sampling

In [4]:
DATA = Path.cwd().parent.joinpath("tests/data")
DATA

PosixPath('/home/cat/projects/MCCE_Scikit/tests/data')

In [5]:
!ls {DATA}

head3.lst  ms_out  run.prm.record  step2_out.pdb


In [6]:
# filepaths of inputs used by MS class:
h3_path = DATA.joinpath("head3.lst")
mcce_output_path = h3_path.parent
mcce_output_path

step2_path = mcce_output_path.joinpath("step2_out.pdb")
msout_path = mcce_output_path.joinpath("ms_out")
msout_path, msout_path.is_dir()

PosixPath('/home/cat/projects/MCCE_Scikit/tests/data')

(PosixPath('/home/cat/projects/MCCE_Scikit/tests/data/ms_out'), True)

In [26]:
opath = None
(not opath) or opath is None

True

In [7]:
# filepaths of outputs:

pH = 5.0
Eh= 0.0
msout_file = io.get_msout_filename(mcce_output_path, pH, Eh)
msout_file

msout_file_dir = msout_file.parent.joinpath(msout_file.stem)
msout_file_dir

PosixPath('/home/cat/projects/MCCE_Scikit/tests/data/ms_out/pH5eH0ms.txt')

PosixPath('/home/cat/projects/MCCE_Scikit/tests/data/ms_out/pH5eH0ms')

In [8]:
start_time = time.time()

io.split_msout_file(mcce_output_path, pH, Eh)

end_time = time.time()
print("io.divide_msout_file() took {:.2f} mins".format((end_time - start_time)/60))

The ms_out file is already split into header and MCi files. Set `overwrite` to True to replace them.
io.divide_msout_file() took 0.00 mins


In [9]:
!ls {msout_file_dir}

MC0  MC1  MC2  MC3  MC4  MC5  header  pdbs_from_ms


In [10]:
pdbs_dir = msout_file_dir.joinpath("pdbs_from_ms")

In [11]:
!ls {pdbs_dir}

mc0_ms1.pdb  mc0_ms114217.pdb  mc0_ms230849.pdb  mc0_ms349302.pdb


# base.MC class

In [12]:
print(base.MS.__doc__)
print(base.MS.__init__.__doc__)

Uses split ms_out files.
MS.init

        Parameters:
            mcce_output_path (str): A MCCE simulation output folder.
            pH (int or float): A pH point.
            Eh (int or float): A Eh point.
            selected_MC (int): The index of an MC run; one of `range(constants.MONTERUNS)`.
            overwrite_split_files (bool): whether to redo the splitting of msout_file.
        


In [14]:
# create instance
start_time = time.time()

ms = base.MS(mcce_output_path, pH, Eh)

d = time.time() - start_time
print(f"Loading of base.MS instance took {d/60:.2f} mins or {d:.2f} seconds")
print(ms)

Loading of base.MS instance took 0.14 mins or 8.56 seconds
MS("/home/cat/projects/MCCE_Scikit/tests/data", 5.0, 0.0, selected_MC=0, overwrite_split_files=False)


In [15]:
# Public vars in MC:
fdir(ms)

['Eh',
 'T',
 'confnames_by_iconfs',
 'conformers',
 'counts',
 'fixed_crg',
 'fixed_iconfs',
 'fixed_ne',
 'fixed_nh',
 'fixed_residue_names',
 'fname',
 'free_residue_names',
 'free_residues',
 'get_occ',
 'iconf_by_confname',
 'ires_by_iconf',
 'mcce_out',
 'method',
 'microstates',
 'msout_file_dir',
 'overwrite_split_files',
 'pH',
 'select_by_conformer',
 'select_by_energy',
 'selected_MC']

In [16]:
print(f"ms.counts : {ms.counts:,}")

ms.counts : 600,000


# ms_sampling module

In [17]:
fdir(sampling)

['Path',
 'base',
 'datetime',
 'get_pdb_remark',
 'get_selected_confs',
 'io',
 'np',
 'pdbs_from_ms_samples',
 'sample_microstates',
 'sampling',
 'sort_microstate_list']

In [20]:
n_sample_size = 4
ms_sort_by = "energy"
output_dir = msout_file_dir
mc_run = ms.selected_MC  # part of pdb name
mc_run

0

In [23]:
# create pdbs from samples ms
start_time = time.time()

sampling.pdbs_from_ms_samples(ms,
                              mcce_output_path,
                              n_sample_size,
                              ms_sort_by,
                              output_dir,
                              list_files=True)

d = time.time() - start_time
print(f"`sampling.pdbs_from_ms_samples` with sample size={n_sample_size:,} took {d/60:.2f} mins or {d:.2f} seconds")

Creating n=4 MCCE_PDB files in /home/cat/projects/MCCE_Scikit/tests/data/ms_out/pH5eH0ms from (n) microstates sorted by 'energy'.
 NOTE: the output pdb will be free of any water molecules in step2_out.pdb.
PDB files creation over.
Files in /home/cat/projects/MCCE_Scikit/tests/data/ms_out/pH5eH0ms/pdbs_from_ms:

	 /home/cat/projects/MCCE_Scikit/tests/data/ms_out/pH5eH0ms/pdbs_from_ms/mc0_ms230849.pdb
	 /home/cat/projects/MCCE_Scikit/tests/data/ms_out/pH5eH0ms/pdbs_from_ms/mc0_ms114217.pdb
	 /home/cat/projects/MCCE_Scikit/tests/data/ms_out/pH5eH0ms/pdbs_from_ms/mc0_ms1.pdb
	 /home/cat/projects/MCCE_Scikit/tests/data/ms_out/pH5eH0ms/pdbs_from_ms/mc0_ms349302.pdb
`sampling.pdbs_from_ms_samples` with sample size=4 took 0.01 mins or 0.90 seconds


In [22]:
!head -n 20 ../tests/data/ms_out/pH5eH0ms/pdbs_from_ms/mc0_ms1.pdb


REMARK 250
REMARK 250 EXPERIMENTAL DETAILS
REMARK 250   EXPERIMENT TYPE               : MCCE simulation
REMARK 250   DATE OF DATA COLLECTION       : 26-Oct-23
REMARK 250   REMARK: DATE OF DATA COLLECTION is the date this pdb was created.
REMARK 250 EXPERIMENTAL CONDITIONS
REMARK 250   TEMPERATURE                   : 298.15 (K)
REMARK 250   PH                            : 5.00
REMARK 250   EH                            : 0.00
REMARK 250   METHOD                        : MONTERUNS
REMARK 250   SELECTED MONTERUN             : 0
REMARK 250   SELECTED MICROSTATE INDEX     : 1
REMARK 250   SELECTED MICROSTATE ENERGY    : 202.89 (kcal/mol)
REMARK 250
ATOM      1  CA  NTR A0001_001   2.696   5.785  12.711   2.000       0.001      01O000M000 
ATOM      2  HA  NTR A0001_001   3.149   5.444  11.801   0.000       0.000      01O000M000 
ATOM      3  N   NTR A0001_001   2.812   4.829  13.856   1.500      -0.003      01O000M000 
ATOM      4  H   NTR A0001_001   2.419   3.912  13.784   1.000       0.